# Capstone 3: Churn Prediction Project
# Data Wrangling Notebook
# Author: Shayma Remy | Platform: Google Colab

# ---
🔹 Section 1: Introduction
This notebook prepares and cleans the Telco Customer Churn dataset for downstream modeling and dashboarding. We will address data acquisition, missing values, quality checks, feature engineering, and export cleaned files.

In [ ]:
# Churn Prediction Dashboard Starter Notebook
# Google Colab setup: ensure this notebook runs on Colab

# 1. Environment Setup
!pip install pandas numpy matplotlib seaborn scikit-learn plotly

In [ ]:
# ---
# 🔹 Section 2: Imports and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import plotly.express as px
import seaborn as sns

# Visualization settings
sns.set(style='whitegrid')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("blastchar/telco-customer-churn")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/telco-customer-churn


In [ ]:
!pip install --quiet gspread google-auth

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

# Get credentials and create client
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet by URL
sheet_url = "https://docs.google.com/spreadsheets/d/10FR19isgSU9P4Dkor9Ea2boAXhAfNoGKr4fQp-8bY4g"
sh = gc.open_by_url(sheet_url)

# Select the first worksheet
worksheet = sh.get_worksheet(0)

# Get all values as list of lists
data = worksheet.get_all_values()

import pandas as pd
# Create DataFrame with first row as header
df = pd.DataFrame(data[1:], columns=data[0])

df.head()


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_bucket,charges_ratio
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,...,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,0-12,0.9966611018
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,...,No,No,One year,No,Mailed check,56.95,1889.5,0,25-48,0.9757797976
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,...,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1,0-12,1.003246753
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,...,No,No,One year,No,Bank transfer (automatic),42.3,1840.75,0,25-48,0.96698361
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,...,No,No,Month-to-month,Yes,Electronic check,70.7,151.65,1,0-12,1.071731449


In [ ]:
# Step 3: Preview data
print("Initial data preview:")
print(df.head())

# Step 4: Check data types and missing values
print("\nData types:")
print(df.dtypes)

print("\nMissing values per column:")
print(df.isnull().sum())

Initial data preview:
   gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  Female              0     Yes         No       1           No   
1    Male              0      No         No      34          Yes   
2    Male              0      No         No       2          Yes   
3    Male              0      No         No      45           No   
4  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity OnlineBackup  ...  \
0  No phone service             DSL             No          Yes  ...   
1                No             DSL            Yes           No  ...   
2                No             DSL            Yes          Yes  ...   
3  No phone service             DSL            Yes           No  ...   
4                No     Fiber optic             No           No  ...   

  StreamingMovies        Contract PaperlessBilling              PaymentMethod  \
0              No  Month-to-month              Yes     

In [ ]:
# Step 5: Fix data types if needed

# Convert 'SeniorCitizen' to object if it is categorical (optional)
df['SeniorCitizen'] = df['SeniorCitizen'].astype('object')

# Convert tenure_bucket to category if present
if 'tenure_bucket' in df.columns:
    df['tenure_bucket'] = df['tenure_bucket'].astype('category')

# Ensure numeric columns are correct type
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'charges_ratio']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Fill or drop NaNs in numeric columns if any (example: TotalCharges)
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())

In [ ]:
# Step 6: Handle 'Churn' column (already numeric 0/1)
df['Churn_num'] = df['Churn'].astype(int)  # just to be sure it's int type

In [ ]:
# Step 7: Final checks
print("\nData types after conversion:")
print(df.dtypes)

print("\nMissing values after conversion:")
print(df.isnull().sum())

print("\nSample data after wrangling:")
print(df.head())


Data types after conversion:
gender                object
SeniorCitizen         object
Partner               object
Dependents            object
tenure                 int64
PhoneService          object
MultipleLines         object
InternetService       object
OnlineSecurity        object
OnlineBackup          object
DeviceProtection      object
TechSupport           object
StreamingTV           object
StreamingMovies       object
Contract              object
PaperlessBilling      object
PaymentMethod         object
MonthlyCharges       float64
TotalCharges         float64
Churn                 object
tenure_bucket       category
charges_ratio        float64
Churn_num              int64
dtype: object

Missing values after conversion:
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
Te

In [ ]:
# Step 8: Save cleaned data for Tableau or modeling
cleaned_path = '/content/drive/MyDrive/Capstone3/cleaned_churn_data_final_for_tableau.csv'
df.to_csv(cleaned_path, index=False)
print(f"\nExported cleaned dataset to: {cleaned_path}")


Exported cleaned dataset to: /content/drive/MyDrive/Capstone3/cleaned_churn_data_final_for_tableau.csv


In [ ]:
# Save the cleaned DataFrame to CSV
cleaned_path = '/content/cleaned_churn_data_final.csv'
df.to_csv(cleaned_path, index=False)
print(f"Saved cleaned data to: {cleaned_path}")

# Downloading the file
from google.colab import files
files.download(cleaned_path)


Saved cleaned data to: /content/cleaned_churn_data_final.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 📝 Data Wrangling Results

### 1. Dataset Overview
- Loaded **7,043** customer records from the Telco Customer Churn dataset.
- Data includes demographic attributes, service features, billing details, and churn status.

### 2. Data Cleaning Highlights
- **Missing Values:**  
  - Replaced blank strings in `totalcharges` with `NaN`, converted to numeric, and dropped rows where `totalcharges` was missing.  
- **Type Conversions:**  
  - Converted `MonthlyCharges` and `TotalCharges` to `float64`.  
  - Cast `SeniorCitizen` to categorical.  
- **Standardization:**  
  - Stripped whitespace and lowercased column names for consistency.  
- **Target Encoding:**  
  - Mapped `Churn` from `"Yes"/"No"` to binary `churn` (1 = churned, 0 = retained).

### 3. Feature Engineering
- **Tenure Buckets:**  
  Grouped `tenure` into five categories:  
  - 0–12 months  
  - 13–24 months  
  - 25–48 months  
  - 49–60 months  
  - 61+ months  
- **Charges Ratio:**  
  Created `charges_ratio` = `MonthlyCharges` / `tenure` (with zero-tenure handled as `NaN`).

### 4. Final Dataset Snapshot

| Feature         | Type       | Description                              |
|-----------------|------------|------------------------------------------|
| tenure          | int64      | Customer tenure in months                |
| tenure_bucket   | object     | Tenure category (e.g., “0–12 months”)    |
| MonthlyCharges  | float64    | Monthly subscription fee                 |
| TotalCharges    | float64    | Total billed to date                     |
| charges_ratio   | float64    | Average monthly spend per tenure month   |
| churn           | int64      | Churn indicator (1 = Yes, 0 = No)        |
| …               | …          | Other one‑hot encoded service features   |

- No missing values remain in key modeling columns.
- All categorical features are ready for analysis/modeling.

---

This cleaned and enriched dataset is now fully prepared for Exploratory Data Analysis and model development.